In [1]:
import subprocess as sp
import sys,os,shutil,pickle,time
from os.path import join
from os.path import expanduser


In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('samsungcamera.csv')
df.head(5)

,stars,comment
0,5,\n\n\n\n\n\n\n\n\n\n \n \n \n This is my...
1,5,Product delievery was late by one day but no i...
2,5,I was in a confusion to buy this or go for one...
3,5,This time Samsung really try to do some thing ...
4,5,1.Do not remove the pre applied screen protect...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   stars    990 non-null    int64 
 1   comment  990 non-null    object
dtypes: int64(1), object(1)
memory usage: 15.6+ KB


In [5]:
df.describe().round(1)

,stars
count,990.0
mean,4.5
std,1.2
min,1.0
25%,5.0
50%,5.0
75%,5.0
max,5.0


In [6]:
# Determine how many missing values exist in the collection, in which case you can use .sum() chained onto is.na()
null_values=df.isna().sum()
null_values=pd.DataFrame(null_values,columns=['null'])
sum_tot=len(df)
null_values['percent']=null_values['null']/sum_tot*100
round(null_values,3).sort_values('percent',ascending=False)

,null,percent
stars,0,0.0
comment,0,0.0


In [7]:
df= df.dropna()
df.shape

(990, 2)

In [8]:
#pip install -q wordcloud
import wordcloud
from nltk.corpus import stopwords
import nltk
import string
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nalinjindal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/nalinjindal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/nalinjindal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/nalinjindal/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [9]:
df['stopwords'] = df['comment'].apply(lambda x: len([x for x in x.split() if x in stop]))
df[['comment','stopwords']].head()

,comment,stopwords
0,\n\n\n\n\n\n\n\n\n\n \n \n \n This is my...,19
1,Product delievery was late by one day but no i...,151
2,I was in a confusion to buy this or go for one...,194
3,This time Samsung really try to do some thing ...,46
4,1.Do not remove the pre applied screen protect...,58


In [10]:
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return count

df['punctuation'] = df['comment'].apply(lambda x: count_punct(x))

In [11]:
df[['comment','punctuation']].head()

,comment,punctuation
0,\n\n\n\n\n\n\n\n\n\n \n \n \n This is my...,2
1,Product delievery was late by one day but no i...,59
2,I was in a confusion to buy this or go for one...,75
3,This time Samsung really try to do some thing ...,16
4,1.Do not remove the pre applied screen protect...,37


In [12]:
df['hastags'] = df['comment'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
df[['comment','hastags']].head()

,comment,hastags
0,\n\n\n\n\n\n\n\n\n\n \n \n \n This is my...,0
1,Product delievery was late by one day but no i...,0
2,I was in a confusion to buy this or go for one...,0
3,This time Samsung really try to do some thing ...,0
4,1.Do not remove the pre applied screen protect...,0


In [13]:
df.hastags.loc[df.hastags != 0].count()

0

In [14]:
df['numerics'] = df['comment'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
df[['comment','numerics']].head()

,comment,numerics
0,\n\n\n\n\n\n\n\n\n\n \n \n \n This is my...,0
1,Product delievery was late by one day but no i...,8
2,I was in a confusion to buy this or go for one...,3
3,This time Samsung really try to do some thing ...,2
4,1.Do not remove the pre applied screen protect...,0


In [15]:
df['upper'] = df['comment'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
df[['comment','upper']].head()

,comment,upper
0,\n\n\n\n\n\n\n\n\n\n \n \n \n This is my...,1
1,Product delievery was late by one day but no i...,2
2,I was in a confusion to buy this or go for one...,29
3,This time Samsung really try to do some thing ...,1
4,1.Do not remove the pre applied screen protect...,0


In [16]:
df['comment'] = df['comment'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['comment'].head()

0    this is my first phone from samsung after appl...
1    product delievery was late by one day but no i...
2    i was in a confusion to buy this or go for one...
3    this time samsung really try to do some thing ...
4    1.do not remove the pre applied screen protect...
Name: comment, dtype: object

In [17]:
df['comment'] = df['comment'].str.replace('[^\w\s]','')
df['comment'].head()

<ipython-input-17-66a30b7fedcd>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['comment'] = df['comment'].str.replace('[^\w\s]','')


0    this is my first phone from samsung after appl...
1    product delievery was late by one day but no i...
2    i was in a confusion to buy this or go for one...
3    this time samsung really try to do some thing ...
4    1do not remove the pre applied screen protecto...
Name: comment, dtype: object

In [18]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['comment'] = df['comment'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['comment'].sample(10)

438                    good somehow camera disappointing
85     worth money price high waste money camera qual...
288                    good somehow camera disappointing
95     1do remove pre applied screen protector doesnt...
401    product delievery late one day issues thanks a...
563    time samsung really try thing new special seri...
492    confusion buy go oneplus 88pro confusion im sh...
602    confusion buy go oneplus 88pro confusion im sh...
756    guys lets cut crap lets get straight onto revi...
510    first phone samsung apple x phone great compar...
Name: comment, dtype: object

In [19]:
def remove_url(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

In [20]:
# remove all urls from df
import re
import string

df['comment'] = df['comment'].apply(lambda x: remove_url(x))

In [21]:
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

In [22]:
# remove all html tags from df
df['comment'] = df['comment'].apply(lambda x: remove_html(x))

In [23]:
# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags 
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [24]:
#Example
remove_emoji("Omg another Earthquake 😔😔")

'Omg another Earthquake '

In [25]:
# remove all emojis from df
df['comment'] = df['comment'].apply(lambda x: remove_emoji(x))

In [26]:
!pip install emot

Defaulting to user installation because normal site-packages is not writeable


In [27]:
from emot.emo_unicode import UNICODE_EMO, EMOTICONS

# Function for removing emoticons
def remove_emoticons(text):
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
    return emoticon_pattern.sub(r'', text)

In [28]:
#Example
remove_emoticons("Hello :-)")

'Hello '

In [29]:
df['comment'] = df['comment'].apply(lambda x: remove_emoticons(x))

In [30]:
from textblob import TextBlob
df['comment'][:5].apply(lambda x: str(TextBlob(x).correct()))

0    first phone samson apple x phone great compare...
1    product delivery late one day issues thanks am...
2    confusion buy go onerous pro confusion in shar...
3    time samson really try thing new special serie...
4    do remove pre applied screen protector doesn c...
Name: comment, dtype: object

In [31]:
# We could do some of the cleaning steps as a sum of opreation like this:

# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [32]:
df['comment'] = df.comment.apply(round1)
df.comment

0      first phone samsung apple x phone great compar...
1      product delievery late one day issues thanks a...
2      confusion buy go oneplus  confusion im sharing...
3      time samsung really try thing new special seri...
4       remove pre applied screen protector doesnt co...
                             ...                        
985    worth money price high waste money camera qual...
986    guys lets cut crap lets get straight onto revi...
987     perfect smartphone  reasonable price point ph...
988                    good somehow camera disappointing
989    awesome product samsungprosvery good audio qua...
Name: comment, Length: 990, dtype: object

In [33]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [34]:
df['comment'] = df.comment.apply(round2)
df.comment

0      first phone samsung apple x phone great compar...
1      product delievery late one day issues thanks a...
2      confusion buy go oneplus  confusion im sharing...
3      time samsung really try thing new special seri...
4       remove pre applied screen protector doesnt co...
                             ...                        
985    worth money price high waste money camera qual...
986    guys lets cut crap lets get straight onto revi...
987     perfect smartphone  reasonable price point ph...
988                    good somehow camera disappointing
989    awesome product samsungprosvery good audio qua...
Name: comment, Length: 990, dtype: object

In [35]:
freq = pd.Series(' '.join(df['comment']).split()).value_counts()[:20]
freq

samsung    2079
one        1980
battery    1881
phone      1881
camera     1485
quality    1386
screen     1287
get         792
day         792
plus        792
good        792
best        693
fast        693
great       693
display     594
rate        594
like        495
exynos      495
size        495
price       495
dtype: int64

In [36]:
df['word_count'] = df['comment'].apply(lambda x: len(str(x).split(" ")))
df[['comment','word_count']].head()

,comment,word_count
0,first phone samsung apple x phone great compar...,26
1,product delievery late one day issues thanks a...,252
2,confusion buy go oneplus confusion im sharing...,249
3,time samsung really try thing new special seri...,85
4,remove pre applied screen protector doesnt co...,102


In [37]:
null_values=df.isna().sum()
null_values=pd.DataFrame(null_values,columns=['null'])
sum_tot=len(df)
null_values['percent']=null_values['null']/sum_tot*100
round(null_values,3).sort_values('percent',ascending=False)

,null,percent
stars,0,0.0
comment,0,0.0
stopwords,0,0.0
punctuation,0,0.0
hastags,0,0.0
numerics,0,0.0
upper,0,0.0
word_count,0,0.0


In [38]:
df['char_count'] = df['comment'].str.len() ## this also includes spaces
df[['comment','char_count']].head()

,comment,char_count
0,first phone samsung apple x phone great compar...,177
1,product delievery late one day issues thanks a...,1530
2,confusion buy go oneplus confusion im sharing...,1535
3,time samsung really try thing new special seri...,516
4,remove pre applied screen protector doesnt co...,564


In [39]:
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/(len(words)+0.000001))

In [40]:
df['avg_word'] = df['comment'].apply(lambda x: avg_word(x)).round(1)
df[['comment','avg_word']].head()

,comment,avg_word
0,first phone samsung apple x phone great compar...,5.8
1,product delievery late one day issues thanks a...,5.4
2,confusion buy go oneplus confusion im sharing...,5.6
3,time samsung really try thing new special seri...,5.6
4,remove pre applied screen protector doesnt co...,5.2


In [41]:
df.sample(2)

,stars,comment,stopwords,punctuation,hastags,numerics,upper,word_count,char_count,avg_word
4,5,remove pre applied screen protector doesnt co...,58,37,0,0,0,102,564,5.2
350,5,first phone samsung apple x phone great compar...,19,2,0,0,1,26,177,5.8


In [42]:
list(df)

['stars',
 'comment',
 'stopwords',
 'punctuation',
 'hastags',
 'numerics',
 'upper',
 'word_count',
 'char_count',
 'avg_word']

In [43]:
df.sample(5)

,stars,comment,stopwords,punctuation,hastags,numerics,upper,word_count,char_count,avg_word
449,5,awesome product samsungprosvery good audio qua...,42,12,0,0,0,68,557,7.4
139,5,awesome product samsungprosvery good audio qua...,42,12,0,0,0,68,557,7.4
311,5,product delievery late one day issues thanks a...,151,59,0,8,2,252,1530,5.4
365,1,worth money price high waste money camera qual...,5,7,0,0,0,12,77,5.5
398,4,good somehow camera disappointing,3,0,0,0,0,4,33,7.5


In [55]:
add_words = ["get",     
"use",     
"first",      
"like","u","goes","im","exynos"     
]

stop_words = set(stopwords.words("english"))
stop_added = stop_words.union(add_words)

In [56]:
df['comment'] = df['comment'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_added))
df['comment'].sample(10)

465    worth money price high waste money camera qual...
931    product delievery late one day issues thanks a...
923    time samsung really try thing new special seri...
85     worth money price high waste money camera qual...
886    guys lets cut crap lets straight onto review d...
113    time samsung really try thing new special seri...
697    perfect smartphone reasonable price point phon...
769    awesome product samsungprosvery good audio qua...
6      guys lets cut crap lets straight onto review d...
308                    good somehow camera disappointing
Name: comment, dtype: object

In [57]:
df1= df

In [58]:
mask = df1.comment.str.endswith('br') 
df1.loc[mask, 'comment'] = df1.loc[mask, 'comment'].str[:-2]

In [59]:
df1['comment'] = df1['comment'].str.rstrip('tty')

In [60]:
df1['comment'].apply(lambda x: x[:-2] if x.endswith('tty') else x)

0      phone samsung apple x phone great compared dis...
1      product delievery late one day issues thanks a...
2      confusion buy go oneplus confusion sharing rev...
3      time samsung really try thing new special seri...
4      remove pre applied screen protector doesnt com...
                             ...                        
985    worth money price high waste money camera qual...
986    guys lets cut crap lets straight onto review d...
987    perfect smartphone reasonable price point phon...
988                    good somehow camera disappointing
989    awesome product samsungprosvery good audio qua...
Name: comment, Length: 990, dtype: object

In [61]:
df1.loc[df1.comment.str.endswith('br'), 'comment']


Series([], Name: comment, dtype: object)

In [62]:
df1.loc[df1.punctuation >= 1000].comment.tolist()

[]

In [63]:
df.loc[df.punctuation >= 1000].comment.tolist()


[]

In [64]:
freq = pd.Series(' '.join(df['comment']).split()).value_counts()[:50]
freq

samsung        2079
one            1980
battery        1881
phone          1881
camera         1485
quality        1386
screen         1287
day             792
plus            792
good            792
best            693
great           693
display         594
rate            594
fast            594
still           495
charging        495
wireless        495
fingerprint     495
price           495
better          495
apple           495
size            495
refresh         495
hours           396
updates         396
option          396
think           396
cloud           396
sensor          396
series          396
say             396
features        396
much            396
life            396
heating         396
market          396
os              396
even            396
feel            396
people          396
using           396
last            396
year            396
dont            396
nice            396
full            396
really          396
look            297
light           297


In [65]:
df.to_csv('samsungcamera2.csv', index=False)